## User Overview Analysis

In [5]:
import sqlalchemy
import pandas as pd
from dotenv import dotenv_values
from sqlalchemy import create_engine
from urllib.parse import quote_plus
from pathlib import Path

### Seting datbase connection requirnments

In [6]:
env_path = Path('.') / '.env'
env_values = dotenv_values(dotenv_path=env_path)

db_password = env_values["POSTGRES_PASSWORD"]
encoded_password = quote_plus(db_password)
database = env_values["POSTGRES_DB"]
user = env_values["POSTGRES_USER"]
server = env_values["POSTGRES_SERVER"]

In [7]:
# checking the connection to the database
engine = create_engine(f'postgresql://{user}:{encoded_password}@{server}/{database}')
try:
    with engine.connect() as connection_str:
        print('Successfully connected to the PostgreSQL database')
except Exception as ex:
    print(f'Sorry failed to connect: {ex}')

Successfully connected to the PostgreSQL database
